# 1ST PART

In [1]:
#imports
from selenium import webdriver 
from selenium.webdriver.chrome.service import Service as ChromeService 
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.selenium_manager import SeleniumManager 
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from time import sleep
from googletrans import Translator
from google_trans_new import google_translator  
from deep_translator import GoogleTranslator
import os
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import pandas as pd
from sqlalchemy import create_engine
import warnings
import sys
from pyspark.sql import SparkSession
import pyspark.sql.functions as sf
# from pyspark.sql.functions import col,udf,lit,split
from pyspark.sql.types import IntegerType, StringType
from tqdm import tqdm
import pyspark.pandas as ps
import pandas as pd
import time
import os
import urllib.request
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

import warnings
warnings.filterwarnings("ignore")

from tqdm import tqdm
import pdfplumber

import subprocess
import pathlib
import ghostscript as gs

warnings.filterwarnings('ignore')

c:\Users\ayush\anaconda3\envs\morph\lib\site-packages\pyspark\pandas\__init__.py:50: UserWarning: 'PYARROW_IGNORE_TIMEZONE' environment variable was not set. It is required to set this environment variable to '1' in both driver and executor sides if you use pyarrow>=2.0.0. pandas-on-Spark will set it for you but it does not work if there is a Spark context already launched.
  warnings.warn(


In [2]:
#Functions
def translator(s,source):
    return GoogleTranslator(source=source, target='en').translate(s)


def translator_v2(s,source='auto'):
        translator = Translator()
        return translator.translate(s, src='hi',dest='en').text if s != '' else 'null' 

translate_udf = sf.udf(translator_v2,StringType())

In [3]:
#selenium webdriver
input_params_dropdown_by_id = {}
input_params_dropdown_by_id['dbselect'] = '2023' #Select Year
input_params_dropdown_by_id['district_id'] = 'मुंबई उपनगर' #District
input_params_dropdown_by_id['taluka_id'] = 'अंधेरी' #Taluka
input_params_dropdown_by_id['village_id'] = 'बांद्रा' #Village

input_params_text_by_id = {}
input_params_text_by_id['free_text'] = '2023' 

url = 'https://pay2igr.igrmaharashtra.gov.in/eDisplay/Propertydetails/index' 
driver = webdriver.Chrome() 
driver.implicitly_wait(10)
driver.get(url)

for key, value in input_params_dropdown_by_id.items():
	print(key,value)
	dropdown = driver.find_element(by=By.ID,value=key)
	dropdown_select = Select(dropdown)
	for option in dropdown_select.options:
		if option.text == value:
			option.click()
			print(f'Clicked {option.text} for {key}')
			break
	driver.implicitly_wait(2)

#Input reg year
driver.find_element(by=By.ID, value='free_text').send_keys(input_params_text_by_id['free_text'])
driver.implicitly_wait(2)

#Input Captcha
captcha = input()
driver.find_element(by=By.ID, value='cpatchaTextBox').send_keys(captcha)
print(f'Entered Captcha {captcha}')
driver.implicitly_wait(2)

#Submit to get result set
driver.find_element(by=By.ID,value='submit').click()
driver.implicitly_wait(10)


#Click on 50 Pages
dropdown = driver.find_element(by=By.NAME, value='tableparty_length')
dropdown_select = Select(dropdown)
for option in dropdown_select.options:
	if option.text == 'All':
		option.click()
		print('Set to All Records')
		break

print('So Far, So Good')

dbselect 2023
Clicked 2023 for dbselect
district_id मुंबई उपनगर
Clicked मुंबई उपनगर for district_id
taluka_id अंधेरी
Clicked अंधेरी for taluka_id
village_id बांद्रा
Clicked बांद्रा for village_id
Entered Captcha 9XVW1
Set to All Records
So Far, So Good


In [4]:
#scrap records
records_raw = pd.DataFrame(columns=['अनु क्र.','दस्त क्र.','दस्त प्रकार','दू. नि. कार्यालय','वर्ष','लिहून देणार','लिहून घेणार','इतर माहीती','सूची क्र. २'])

print('Data Loading...')
for index, table in enumerate(tqdm(driver.find_elements(by=By.ID, value='tbdata'))):
    data = [item.text if item.text != 'सूची क्र. २' else item.find_element(by=By.TAG_NAME,value='a').get_attribute('href') for item in table.find_elements(by=By.XPATH, value=".//*[self::td or self::th]")]
    records_raw.loc[len(records_raw)] = data
    # print(data)
    # print(f'{index} Row appended',)

print('Finished!')
records_raw
records_translate = records_raw.copy()

Data Loading...


100%|██████████| 1209/1209 [03:17<00:00,  6.11it/s]

Finished!


In [5]:
# Insert Scraped Raw Data in postgres
db_user = 'postgres'
db_password = 'Sunrise12345'
db_host = '127.0.0.1'
db_port = '5432'
db_name = 'propReturns'

engine = create_engine(f'postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}')
table_name = 'record_details_raw'

records_raw.to_sql(table_name, engine, if_exists='replace', index=True)

print(f"{len(records_raw)} DataFrame has been inserted into the '{table_name}' table.")

1209 DataFrame has been inserted into the 'record_details_raw' table.


In [6]:
records_translate = records_raw.copy()
records_translate.columns = [translator(col,source='auto') for col in records_translate.columns]
cols_to_translate = [
    'diarrhea type',
    'Du. Prohibit. Office',
    'Will write', 
    'Will write down', 
    'Other information'
    ]

In [7]:
records_translate.columns

Index(['Sl no.', 'Diarrhea no.', 'diarrhea type', 'Du. Prohibit. Office',
       'Year', 'Will write', 'Will write down', 'Other information',
       'List no. 2'],
      dtype='object')

In [8]:
records_translate['to_translate'] = records_translate['diarrhea type'] + '__$__' \
                                    + records_translate['Du. Prohibit. Office'] + '__$__' \
                                    + records_translate['Will write'] + '__$__' \
                                    + records_translate['Will write down'] + '__$__' \
                                    + records_translate['Other information']  

In [9]:
records_translated = pd.DataFrame(columns=records_translate.columns)
batch_size = 1
start = 0
for batch in tqdm(range(start,len(records_translate))):
    # print(f'Batch -> [{batch},{batch+batch_size}]')
    temp_df = records_translate.iloc[batch:batch+batch_size,:].copy()
    temp_df['to_translate'] = temp_df['to_translate'].apply(translator_v2)
    # for col in cols_to_translate:
    #     # print(f'Translating {col}')
    #     temp_df[col] = temp_df[col].apply(lambda x: translator(x,source='hi'))
    #     # print(f'Done')
    records_translated = pd.concat([records_translated,temp_df])
records_translated.rename(columns={'Will write':'buyer_name','Will write down':'seller_name'},inplace=True)

 85%|████████▌ | 1033/1209 [04:45<00:48,  3.61it/s]


ReadTimeout: The read operation timed out

In [42]:
records_translated_og = records_translated.copy()

In [43]:
cols_to_translate = [
    'diarrhea type',
    'Du. Prohibit. Office',
    'buyer_name', 
    'seller_name', 
    'Other information'
    ]

records_translated[cols_to_translate] = records_translated['to_translate'].apply(lambda x: pd.Series(x.split('__$__')))

In [44]:
records_translated_og

,index,Sl no.,Diarrhea no.,diarrhea type,Du. Prohibit. Office,Year,buyer_name,seller_name,Other information,List no. 2,to_translate
0,0,1,13217,Rent lease,cum d.n. Andheri 7,25/07/2023,"1) Bhavin Sheth, Authorized Trustee on behalf ...","1) Hanmant Dhanvare, Executive Engineer, Mahar...",1) Other information: Piece or parcel of land ...,https://pay2igr.igrmaharashtra.gov.in/eDisplay...,Rent lease__$__cum d.n. Andheri 7__$__1) Bhavi...
1,1,2,4286,Development Agreement,Co. Andheri 7,10/03/2023,1) Ajit Singh Kartar Singh Chandok,"1) Paresh Ranchod Patel, Partner, Navish Realt...","1) Other information: Land and construction, p...",https://pay2igr.igrmaharashtra.gov.in/eDisplay...,Development Agreement__$__Co. Andheri 7__$__1)...
2,2,3,2449,Sale Deed,Co Dn. Andheri 7,09/02/2023,1) Nupur Anil Kalke Mukhtyar Shaila Anil Kalke...,1) Harsh Satpal Malhotra,"1) Other Information: House No: 701, Floor No:...",https://pay2igr.igrmaharashtra.gov.in/eDisplay...,Sale Deed__$__Co Dn. Andheri 7__$__1) Nupur An...
3,3,4,8691,66-Notice of Lis Pendency,cum. Andheri 7,22/05/2023,1) Pradeep Soni,,1) Other Information: City Civil Court at Dind...,https://pay2igr.igrmaharashtra.gov.in/eDisplay...,66-Notice of Lis Pendency__$__cum. Andheri 7__...
4,4,5,3551,65-Mistake Correction Letter,with D.N. Andheri 7,27/02/2023,1) Gunjan Yogeet Kapoor\n2) Tushar Subhash Oberoi,1) Shama Subhash Oberoi,"1) House No: 201, Mala No: 2, Building Boat: O...",https://pay2igr.igrmaharashtra.gov.in/eDisplay...,65-Mistake Correction Letter__$__with D.N. And...
...,...,...,...,...,...,...,...,...,...,...,...
1204,1204,1205,3173,Live Ad Licenses,Co D.N. ANDHERI 7,20/02/2023,1) COELHO NOALA,1) MUHAMMAD KUNHI K C,"1) FLAT NO:35/A, MLA NO:1ST FLOOR, BUILDING NO...",https://pay2igr.igrmaharashtra.gov.in/eDisplay...,Live Ad Licenses__$__Co D.N. ANDHERI 7__$__1) ...
1205,1205,1206,2531,Agreement,Co.D. Andheri 7,13/02/2023,"1) Chandresh Mehta, Director of Keystone Realt...",1) Rashmi Satish Kewalramani.\n2) Varun Satish...,"1) Other Information: Real Estate Project-\""Ru...",https://pay2igr.igrmaharashtra.gov.in/eDisplay...,Agreement__$__Co.D. Andheri 7__$__1) Chandresh...
1206,1206,1207,2863,Live Ad Licenses,Co D.N. Andheri 7,15/02/2023,1) Faisal Abdul Rahim Ghori,1) Jeram Chamadia (HUF) Laxman,"1) Flat No:Garage No.1, Floor No:Ground, Build...",https://pay2igr.igrmaharashtra.gov.in/eDisplay...,Live Ad Licenses__$__Co D.N. Andheri 7__$__1) ...
1207,1207,1208,2506,Release Deed,Co Dn. Andheri 7,10/02/2023,1) Monisha Rahim Harjee alias Monisha B.Charan...,1) Karim B. Charaniya,"1) House No: 1201, Floor No: Barawa Majla, Bui...",https://pay2igr.igrmaharashtra.gov.in/eDisplay...,Release Deed__$__Co Dn. Andheri 7__$__1) Monis...


In [10]:
# del os.environ['PYSPARK_PYTHON'] 
# del os.environ['PYSPARK_DRIVER_PYTHON'] 
os.environ['PYSPARK_PYTHON'] = sys.executable 
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

In [72]:
spark.stop()
del spark

In [11]:
# Approach Via Spark
spark = SparkSession.builder.getOrCreate()
# spark = SparkSession.builder \
#         .config('spark.executor.instances', 4) \
#         .config("spark.sql.execution.arrow.enabled", "true") \
#         .config("spark.dynamicAllocation.enabled", "true") \
#         .config("spark.dynamicAllocation.minExecutors","1") \
#         .config("spark.dynamicAllocation.maxExecutors","5") \
#         .config("spark.executor.cores",6) \
#         .config("spark.executor.memory","2g") \
#         .getOrCreate()
# spark.conf.set("spark.sql.execution.arrow.enabled", "true")
# #spark.conf.set("spark.dynamicAllocation.enabled", "true")
# spark.conf.set("spark.executor.cores", 4)
# spark.conf.set("spark.dynamicAllocation.minExecutors","1")
# spark.conf.set("spark.dynamicAllocation.maxExecutors","5")
# conf = spark.sparkContext.getConf()
spark

In [110]:
del spark_df

In [12]:
#Get the Data from SQL directly
host = '127.0.0.1'
port = '5432'
database = 'propReturns'
username = 'postgres'
password = 'Sunrise12345'
url = f"jdbc:postgresql://{host}:{port}/{database}"
table_name = 'record_details_raw'

engine = create_engine(f'postgresql://{username}:{password}@{host}:{port}/{database}')

spark_df = spark.read.format("jdbc") \
        .option("url", url) \
        .option("driver", "org.postgresql.Driver") \
        .option("dbtable", table_name) \
        .option("user", username) \
        .option("password", password).load()

records_raw = pd.read_sql('record_details_raw',engine)

In [13]:
new_names = []
for col in records_raw.columns:
    new_names.append(translator(col,source='auto'))

new_names = [x.replace('.','') for x in new_names]
spark_df = spark_df.toDF(*new_names)
spark_df = spark_df.withColumnRenamed(existing='Will write',new='buyer_name')
spark_df = spark_df.withColumnRenamed(existing='Will write down',new='seller_name')

In [14]:
spark_df.show()

+-----+-----+-----------+--------------------+------------------+----------+--------------------+--------------------+--------------------+--------------------+
|index|Sl no|Diarrhea no|       diarrhea type|Du Prohibit Office|      Year|          buyer_name|         seller_name|   Other information|           List no 2|
+-----+-----+-----------+--------------------+------------------+----------+--------------------+--------------------+--------------------+--------------------+
|    0|    1|      13217|           भाडेपट्टा|सह दु.नि. अंधेरी 7|25/07/2023|1) श्रीमती चंद्रक...|1) महाराष्ट़ हाऊस...|1) इतर माहिती: पि...|https://pay2igr.i...|
|    1|    2|       4286|       विकसनकरारनामा|सह दु.नि. अंधेरी 7|10/03/2023|1) अजित सिंह करता...|1) नाविश रियल्टी ...|1) इतर माहिती: जम...|https://pay2igr.i...|
|    2|    3|       2449|             सेल डीड|सह दु.नि. अंधेरी 7|09/02/2023|1) नूपुर अनिल कळक...|1) हर्ष सतपाल मल्...|1) इतर माहिती: सद...|https://pay2igr.i...|
|    3|    4|       8691|66-नोटीस 

In [15]:
spark_df = spark_df.withColumn("to_translate",sf.concat(sf.col("diarrhea type"),sf.lit('$'),
                                          sf.col('Du Prohibit Office'),sf.lit('$'),
                                          sf.col('buyer_name'),sf.lit('$'),
                                          sf.col('seller_name'),sf.lit('$'),
                                          sf.col('Other information')))

In [16]:
spark_df = spark_df.withColumn('to_translate',translate_udf(spark_df['to_translate']))

In [17]:
spark_df.withColumn('check',sf.split(spark_df['to_translate'],'\\$').getItem(1)).select('check').show()

+--------------------+
|               check|
+--------------------+
|  cum D.N. Andheri 7|
| with World Healt...|
|   cum Dn. Andheri 7|
|  cum D.N. Andheri 7|
|  cum D.N. Andheri 7|
|       co. Andheri 7|
|  सह दु.नि. अंधेरी 7|
|  cum D.N. Andheri 7|
| with D.O. Andheri 7|
|  cum D.N. Andheri 7|
|  cum D.N. Andheri 7|
|  cum D.N. Andheri 7|
|  cum D.N. Andheri 7|
|   cum Dn. Andheri 7|
|       co. Andheri 7|
|  सह दु.नि. अंधेरी 7|
|   co d.n. Andheri 7|
|   cum Dn. Andheri 7|
|  cum D.N. Andheri 7|
|     co.in Andheri 7|
+--------------------+
only showing top 20 rows



In [21]:
spark_df = spark_df.withColumn('diarrhea type',sf.split(spark_df['to_translate'],'\\$')[0])\
        .withColumn('Du Prohibit Office',sf.split(spark_df['to_translate'],'\\$')[1])\
        .withColumn('buyer_name',sf.split(spark_df['to_translate'],'\\$')[2])\
        .withColumn('seller_name',sf.split(spark_df['to_translate'],'\\$')[3])\
        .withColumn('Other information',sf.split(spark_df['to_translate'],'\\$')[4])

In [23]:
table_name_sink = 'record_details_translated'

spark_df.select("*").write.format("jdbc")\
    .option("url", url) \
    .option("driver", "org.postgresql.Driver") \
    .option("dbtable", table_name_sink) \
    .option("user", username) \
    .option("password", password).save()

# PDF PART

In [11]:
#Get the Data from SQL directly
host = '127.0.0.1'
port = '5432'
database = 'propReturns'
username = 'postgres'
password = 'Sunrise12345'
url = f"jdbc:postgresql://{host}:{port}/{database}"
table_name = 'record_details_translated_v2'

engine = create_engine(f'postgresql://{username}:{password}@{host}:{port}/{database}')

# spark_df = spark.read.format("jdbc") \
#         .option("url", url) \
#         .option("driver", "org.postgresql.Driver") \
#         .option("dbtable", table_name) \
#         .option("user", username) \
#         .option("password", password).load()

records_translated = pd.read_sql('record_details_translated_v2',engine)

In [14]:
records_translated.columns

Index(['index', 'Sl no', 'Diarrhea no', 'diarrhea type', 'Du Prohibit Office',
       'Year', 'buyer_name', 'seller_name', 'Other information', 'List no 2'],
      dtype='object')

In [4]:
import urllib.request
import ssl

# This restores the same behavior as before.
context = ssl._create_unverified_context()
# urllib.urlopen("https://no-valid-cert", context=context)
pdf_path = "https://pay2igr.igrmaharashtra.gov.in/eDisplay/propertydetails/indexii/Mw%3D%3D/MDUxNDA1MzY5OTAwMDAwNDI4NjIwMjNJUw%3D%3D"
def download_file(download_url, filename):
    response = urllib.request.urlopen(download_url,context=context)    
    file = open(filename + ".pdf", 'wb')
    file.write(response.read())
    file.close()
 
download_file(download_url=pdf_path, filename="Test")

In [18]:
#Approach Via Pandas --> Just downloading the files
batch_size = 100
start = 0
end = len(records_translated)
# end = 5
os.makedirs("pdfs", exist_ok = True)
for batch in range(start,end,batch_size):
    print(f'Batch -> [{batch},{batch+batch_size}]')
    temp_df = records_translated.iloc[batch:batch+batch_size,:].copy()
    for index, row in tqdm(temp_df.iterrows()):
        download_file(download_url=row['List no 2'], filename=f"pdfs/{row['Sl no']}")

Batch -> [0,100]


100it [01:39,  1.01it/s]


Batch -> [100,200]


100it [01:45,  1.06s/it]


Batch -> [200,300]


...

100it [01:39,  1.01it/s]


Batch -> [1100,1200]


100it [01:34,  1.06it/s]


Batch -> [1200,1300]


9it [00:08,  1.04it/s]


In [13]:
def curves_to_edges(cs):
    edges = []
    for c in cs:
        edges += pdfplumber.utils.rect_to_edges(c)
    return edges

# Get the bounding boxes of the tables on the page.
def not_within_bboxes(obj):
    """Check if the object is in any of the table's bbox."""
    def obj_in_bbox(_bbox):
        """See https://github.com/jsvine/pdfplumber/blob/stable/pdfplumber/table.py#L404"""
        v_mid = (obj["top"] + obj["bottom"]) / 2
        h_mid = (obj["x0"] + obj["x1"]) / 2
        x0, top, x1, bottom = _bbox
        return (h_mid >= x0) and (h_mid < x1) and (v_mid >= top) and (v_mid < bottom)
    return not any(obj_in_bbox(__bbox) for __bbox in bboxes)


In [33]:
save_path = r'C:\Users\ayush\OneDrive\Desktop\Projects\propreturns_assignment\Mumbai_Real_Estate_Data\pdfs\1.pdf'
pdf = pdfplumber.open(save_path)
page = pdf.pages[0]

table_settings = {
    "vertical_strategy": "explicit",
    "horizontal_strategy": "explicit",
    "explicit_vertical_lines": curves_to_edges(page.curves + page.edges),
    "explicit_horizontal_lines": curves_to_edges(page.curves + page.edges),
    "intersection_y_tolerance": 10,
}

bboxes = [table.bbox for table in page.find_tables(table_settings=table_settings)]

non_table_data = page.filter(not_within_bboxes).extract_text()

table_data_in_nested_list = page.extract_table()
cols = ["sl_no", "pdf_urls", "save_path", "non_table_text", "table_dict"]
extracted_df = pd.DataFrame(columns = cols)
table_data = {li[0]: li[1] for li in table_data_in_nested_list}
curr_df = pd.DataFrame([[0, '', '', non_table_data, table_data]], columns = cols)
extracted_df = pd.concat([extracted_df, curr_df],ignore_index=True)

In [116]:
extracted_df_2 = extracted_df.copy()

In [111]:
def translate_table(non_table_data, table_data):
    visited = False
    temp_dict = {}
    temp_string = ''
    for key, value in table_data.items():
        combined_value = f'{key}__$__{value}' if visited else f'{key}__$__{value}__$__{non_table_data}'
        combined_value = translator_v2(combined_value).split('__$__')
        if not visited:
            visited = True
            temp_string = combined_value[2]
        temp_dict[combined_value[0]] = combined_value[1]
        
    return pd.Series([temp_dict, temp_string])

In [112]:
# extracted_df_2[["table_dict", "non_table_text"]] = 
extracted_df_2[["table_dict", "non_table_text"]].apply(lambda x: translate_table(x["non_table_text"], x["table_dict"]),axis=1)

,0,1
0,{'(1)Subrogation type': 'Development agreement...,4286514 List  . 2 Second Director: Co-Dr. And...


In [117]:
extracted_df_2 = pd.concat([extracted_df_2, extracted_df_2[["table_dict", "non_table_text"]].apply(lambda x: translate_table(x["non_table_text"], x["table_dict"]),axis=1)], axis = 1)

In [124]:
extracted_df_2 = extracted_df_2.rename(columns = {0: "table_data", 1: "non_table_data"})

In [125]:
extracted_df_2

,sl_no,pdf_urls,save_path,non_table_text,table_dict,table_data,non_table_data
0,0,,,4286514 सचू ी  . २ दुयम िनबंधक :सह दु.िन. अं...,"{'(1)दतऐवज कार': 'िवकसनकरारनामा', '(2)मोबदला...",{'(1)Substitute type': 'Development agreement'...,4286514 list . 2 Second Director: Co-Dr. Andhe...
